In [1]:
# !pip install neurokit2

In [2]:
# !pip install data

In [3]:
import os
import numpy as np
import pandas as pd

from matplotlib.ticker import MultipleLocator
import matplotlib.pyplot as plt
from biosppy.signals import ecg
import neurokit2 as nk
from concurrent.futures import ProcessPoolExecutor
import concurrent

import sys
sys.path.append(os.path.abspath('src'))
# from data import load_ecgs_wfdb, load_sample

import pandas as pd

from neurokit2 import ecg_clean
from neurokit2 import ecg_delineate
from neurokit2 import ecg_peaks
from neurokit2 import ecg_phase
from neurokit2 import ecg_quality

def process_ecg(ecg_signal, sampling_rate=500, method="neurokit"):
    try:
        ecg_cleaned = ecg_clean(ecg_signal, sampling_rate=sampling_rate, method=method)
        
        # R-peaks
        instant_peaks, rpeaks, = ecg_peaks(
            ecg_cleaned=ecg_cleaned, sampling_rate=sampling_rate, method="neurokit", correct_artifacts=True
        )

        # quality = ecg_quality(ecg_cleaned, rpeaks=rpeaks['ECG_R_Peaks'], sampling_rate=sampling_rate)

        # Additional info of the ecg signal
        delineate_signal, delineate_info = ecg_delineate(
            ecg_cleaned=ecg_cleaned, rpeaks=rpeaks, sampling_rate=sampling_rate
        )
    except:
        return None, None

        # try:
        #     ecg_out = ecg.ecg(signal=ecg_signal, sampling_rate=500., show=False, interactive=False)
        #     ecg_out = ecg.correct_rpeaks(signal=ecg_signal, rpeaks=ecg_out['rpeaks'], sampling_rate=500.)
        #     ecg_rpeaks = ecg_out['rpeaks']

        #     instant_peaks, rpeaks, = ecg_peaks(
        #         ecg_cleaned=ecg_cleaned, sampling_rate=sampling_rate, method="neurokit", correct_artifacts=True
        #     )

        #     delineate_signal, delineate_info = ecg_delineate(
        #         ecg_cleaned=ecg_cleaned, rpeaks=rpeaks, sampling_rate=sampling_rate
        #     )
        # except:
        #     return None, None


    # Rpeaks location and sampling rate in dict info
    delineate_info = pd.DataFrame(delineate_info)
    delineate_info['R'] = rpeaks['ECG_R_Peaks'].astype(int)

    rename_dict = {
        'ECG_P_Peaks': 'P',
        'ECG_P_Onsets': 'P_on',
        'ECG_P_Offsets': 'P_off',
        'ECG_Q_Peaks': 'Q',
        'ECG_R_Onsets': 'R_on',
        'ECG_R_Offsets': 'R_off',
        'ECG_S_Peaks': 'S',
        'ECG_T_Peaks': 'T',
        'ECG_T_Onsets': 'T_on',
        'ECG_T_Offsets': 'T_off',
    }

    delineate_info = delineate_info.rename(columns=rename_dict)

    return ecg_cleaned, delineate_info

def calculate_features(wf):
    wf = pq.read_table(wf).to_pandas()
    wf = wf.copy()
    # wf = wf[:, 1]
    lead_ii = wf.iloc[:, 1]
    signals, del_waves = process_ecg(lead_ii, sampling_rate=500, method="neurokit")
     # Check if del_waves is None before proceeding
    
    if del_waves is None:
        # Create a DataFrame with one row of NaN values for each column
        columns = ['RR_Int', 'HR', 'PR_Seg', 'PR_Int', 'ST_Seg', 'QT_Int', 
                   'QRS_Dur', 'T_Dur', 'P_Dur', 'HRV', 'SDNN', 'RMSSD', 'QRS_Amp_II', 
                   'ST_Seg_Lvl', 'Tpeak_Tend', 'P_Amp_I', 'Q_Amp_I', 'R_Amp_I', 'S_Amp_I', 
                   'T_Amp_I', 'P_Amp_II', 'Q_Amp_II', 'R_Amp_II', 'S_Amp_II', 'T_Amp_II', 
                   'P_Amp_III', 'Q_Amp_III', 'R_Amp_III', 'S_Amp_III', 'T_Amp_III', 'P_Amp_aVR', 
                   'Q_Amp_aVR', 'R_Amp_aVR', 'S_Amp_aVR', 'T_Amp_aVR', 'P_Amp_aVL', 'Q_Amp_aVL', 
                   'R_Amp_aVL', 'S_Amp_aVL', 'T_Amp_aVL', 'P_Amp_aVF', 'Q_Amp_aVF', 'R_Amp_aVF', 
                   'S_Amp_aVF', 'T_Amp_aVF', 'P_Amp_V1', 'Q_Amp_V1', 'R_Amp_V1', 'S_Amp_V1', 
                   'T_Amp_V1', 'P_Amp_V2', 'Q_Amp_V2', 'R_Amp_V2', 'S_Amp_V2', 'T_Amp_V2', 
                   'P_Amp_V3', 'Q_Amp_V3', 'R_Amp_V3', 'S_Amp_V3', 'T_Amp_V3', 'P_Amp_V4', 
                   'Q_Amp_V4', 'R_Amp_V4', 'S_Amp_V4', 'T_Amp_V4', 'P_Amp_V5', 'Q_Amp_V5', 
                   'R_Amp_V5', 'S_Amp_V5', 'T_Amp_V5', 'P_Amp_V6', 'Q_Amp_V6', 'R_Amp_V6', 
                   'S_Amp_V6', 'T_Amp_V6']
        df_nan = pd.DataFrame([pd.NA] * len(columns)).T
        df_nan.columns = columns
        return df_nan

    # Drop any rows that are missing values
    del_waves = del_waves.dropna()

    # Convert all values to int
    del_waves = del_waves.astype(int)

    features = {}

    sampling_rate = 500.

    # Basic Intervals and Segments
    rr_intervals = np.diff(del_waves['R']) / sampling_rate
    features['RR_Int'] = np.mean(rr_intervals)
    features['HR'] = 60 / features['RR_Int']
    features['PR_Seg'] = np.mean(del_waves['R'] - del_waves['P_off']) / sampling_rate
    features['PR_Int'] = np.mean(del_waves['R'] - del_waves['P_on']) / sampling_rate
    features['ST_Seg'] = np.mean(del_waves['T_on'] - del_waves['S']) / sampling_rate
    features['QT_Int'] = np.mean((del_waves['T_off'] - del_waves['Q']) / sampling_rate)
    features['QRS_Dur'] = np.mean((del_waves['S'] - del_waves['Q']) / sampling_rate)
    features['T_Dur'] = np.mean((del_waves['T_off'] - del_waves['T_on']) / sampling_rate)
    features['P_Dur'] = np.mean((del_waves['P_off'] - del_waves['P_on']) / sampling_rate)

    features['HRV'] = np.std(rr_intervals)
    features['SDNN'] = np.std(rr_intervals)
    features['RMSSD'] = np.sqrt(np.mean(np.square(np.diff(rr_intervals))))

    features['QRS_Amp_II'] = np.mean(lead_ii[del_waves['R']] - lead_ii[del_waves['S']])
    features['ST_Seg_Lvl'] = np.mean(lead_ii[del_waves['T_on']] - lead_ii[del_waves['R_off']])

    features['Tpeak_Tend'] = np.mean((del_waves['T_off'] - del_waves['T']) / sampling_rate)

    leads = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']

    # Amplitude features
    for i, lead in enumerate(leads):
        features[f'P_Amp_{lead}'] = np.mean(wf.iloc[del_waves['P'], i])
        features[f'Q_Amp_{lead}'] = np.mean(wf.iloc[del_waves['Q'], i])
        features[f'R_Amp_{lead}'] = np.mean(wf.iloc[del_waves['R'], i])
        features[f'S_Amp_{lead}'] = np.mean(wf.iloc[del_waves['S'], i])
        features[f'T_Amp_{lead}'] = np.mean(wf.iloc[del_waves['T'], i])

    # Additional Features
    # Heart Rate Variability (HRV)

    # features['R_Peak_Time'] = np.mean(del_waves['R']) / sampling_rate
    # features['S_Depth'] = np.min(lead_ii[del_waves['S']])
    # features['T_Peak_Time'] = np.mean(del_waves['T']) / sampling_rate
    # features['P_Area'] = np.trapz(wf[del_waves['P_on']:del_waves['P_off']], dx=1/sampling_rate)
    features = pd.DataFrame([features])
    
    return features

def calculate_features_batch(ecg_minibatch):
    """
    Calculate features for a batch of ECG signals.

    :param ecg_minibatch: A minibatch of ECG signals with shape (minibatch, 5000, 12).
    :return: A list of dictionaries, each containing features of a corresponding ECG signal.
    """
    features_batch = []
    for wf in ecg_minibatch:
        try:
            features = calculate_features(wf)
        except:
            features = None
        features_batch.append(features)
    return features_batch

def split_into_batches(ecg_batch, batch_size):
    """
    Split the ECG data into batches.

    :param ecg_batch: A batch of ECG signals with shape (b, 5000, 12).
    :param batch_size: Size of each batch.
    :return: A list of batches.
    """
    return [ecg_batch[i:i + batch_size] for i in range(0, len(ecg_batch), batch_size)]


def parallel_process_ecgs(ecg_batch, batch_size=100, n_jobs=16):
    """
    Parallel processing of ECG signals in batches using calculate_features_batch.

    :param ecg_batch: A batch of ECG signals with shape (b, 5000, 12).
    :param batch_size: The size of each batch for processing.
    :param n_jobs: Number of parallel jobs to run.
    :return: List of feature dictionaries for each ECG signal.
    """
    # Split the ECG data into batches
    batches = split_into_batches(ecg_batch, batch_size)
    all_features = []

    print(f"Processing {len(batches)} batches of size {batch_size} with {n_jobs} jobs.")

    with ProcessPoolExecutor(max_workers=n_jobs) as executor:
        # Submit batches to the executor
        futures = [executor.submit(calculate_features_batch, batch) for batch in batches]
        
        # Collect the results as they become available
        for future in concurrent.futures.as_completed(futures):
            all_features.extend(future.result())
    
    return all_features


# if __name__ == '__main__':
#     # all, surgical, surgical_cardiac, surgical_noncardiac, icu, icu_cardiac, icu_noncardiac
#     SAMPLE = 'surgical_noncardiac' #'surgical_noncardiac'
#     SAMPLE_TYPE = '30d_before-day_before' #'after_admission-day_of' # 'all', '30d_before-day_before', '30d_before-day_of', 'after_admission-day_before', 'after_admission-day_of'

#     sample_df = load_sample(SAMPLE, SAMPLE_TYPE)
#     sample_df = sample_df[:10]
#     wf_df = load_ecgs_wfdb(sample_df, batch_size=None)

#     wfs = wf_df['p_signal'].to_numpy()
#     wfs = np.stack(wfs, axis=0)
 
#     wf = wf_df.iloc[0]['p_signal']
#     # fig = plot_ecg(wf, additional_waveforms, additional_labels)
#     feats = calculate_features(wf)

#     # out = parallel_process_ecgs(wfs)

#     # print(out)


In [4]:
import subprocess
import getpass
import os
from SciServer import Authentication
myUserName = Authentication.getKeystoneUserWithToken(Authentication.getToken()).userName
passwd = "westHopkins2442!"

userstring = "username=" + myUserName + ",workgroup=win,uid=idies,password=" + passwd
projectname = "LCICM"
dir = "//cloud.nas.jh.edu/sddesktop$/" + projectname

devnull = open(os.devnull, 'w')
subprocess.run(["sudo", "mkdir", "/home/idies/workspace/SAFE/"], capture_output=False)
subprocess.run(["sudo", "chown", "idies:idies", "/home/idies/workspace/SAFE/"], capture_output=False)
try:
    subprocess.run(["sudo", "mount", "-t", "cifs", dir, "/home/idies/workspace/SAFE/", "-o", userstring], stdout=devnull, stderr=devnull)
except FileNotFoundError as e:
    print(e)

In [5]:
ecg_cleaned, delineate_info = process_ecg('/home/idies/workspace/SAFE/ecg_preprocessed/13:45:00_11_01_2110_18106347')

In [6]:
import pyarrow.parquet as pq

In [7]:
wf = pq.read_table("/home/idies/workspace/SAFE/ecg_preprocessed/01:47:00_08_11_2110_14717666").to_pandas()
print(wf)

          I     II    III    aVR    aVL    aVF     V1     V2     V3     V4  \
0     0.040  0.000 -0.035 -0.025 -0.015  0.035  0.005  0.020  0.005  0.005   
1     0.030 -0.010 -0.035 -0.015 -0.020  0.030  0.005  0.015  0.005  0.005   
2     0.020 -0.020 -0.035 -0.005 -0.025  0.025  0.005  0.015  0.005  0.005   
3     0.020 -0.020 -0.035 -0.005 -0.025  0.025  0.005  0.015  0.005  0.005   
4     0.020 -0.020 -0.035 -0.005 -0.025  0.025  0.005  0.020  0.015  0.005   
...     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
4995 -0.120 -0.075  0.050  0.090 -0.010 -0.085  0.060  0.000  0.000 -0.065   
4996 -0.130 -0.055  0.080  0.085  0.015 -0.105  0.055  0.010  0.020 -0.045   
4997 -0.135 -0.040  0.100  0.080  0.035 -0.120  0.060  0.015  0.040 -0.025   
4998 -0.140 -0.020  0.125  0.075  0.055 -0.135  0.055  0.020  0.045 -0.005   
4999 -0.140  0.000  0.140  0.065  0.075 -0.140  0.045  0.020  0.045  0.005   

         V5     V6  
0    -0.035  0.000  
1    -0.035 -0.005  


In [8]:
wf.iloc[:, 0]

0       0.040
1       0.030
2       0.020
3       0.020
4       0.020
        ...  
4995   -0.120
4996   -0.130
4997   -0.135
4998   -0.140
4999   -0.140
Name: I, Length: 5000, dtype: float64

In [9]:
print(delineate_info)

None


In [10]:
# for HF positive patients
import csv

# import CSV w/ list of all HF positive ECG IDs
pos_ecg_id = []
pos_ecg_csv = open("/home/idies/workspace/SAFE/hf_ecg_list_id.csv", "r")
for row in pos_ecg_csv:
    row = row.strip()
    pos_ecg_id.append(row)
    
pos_ecg_id = pos_ecg_id[1:]
print(len(set(pos_ecg_id)))
print(pos_ecg_id[0:5])
# 5346 total pos HF patients' ECGs

21439
['13:45:00_11_01_2110_18106347', '15:43:00_13_01_2110_16284044', '04:02:00_17_01_2110_18780420', '08:05:00_19_01_2110_13201095', '02:02:00_22_01_2110_16006168']


In [11]:
df = calculate_features("/home/idies/workspace/SAFE/Databases/mimic-iv-ecg-parquet/12839207/23:21:00_30_08_2164_12839207.parquet")
print (df)

   RR_Int         HR    PR_Seg  PR_Int    ST_Seg    QT_Int   QRS_Dur  \
0  1.4288  41.993281  0.120333   0.198  0.005333  0.222333  0.137333   

      T_Dur     P_Dur       HRV  ...  P_Amp_V5  Q_Amp_V5  R_Amp_V5  S_Amp_V5  \
0  0.079667  0.077667  0.907276  ...  0.084167  0.021667  0.211667 -0.278333   

   T_Amp_V5  P_Amp_V6  Q_Amp_V6  R_Amp_V6  S_Amp_V6  T_Amp_V6  
0 -0.006667    0.0575  0.003333  0.313333     -0.33 -0.003333  

[1 rows x 75 columns]


In [12]:
print(len(df))
print(df.keys())

1
Index(['RR_Int', 'HR', 'PR_Seg', 'PR_Int', 'ST_Seg', 'QT_Int', 'QRS_Dur',
       'T_Dur', 'P_Dur', 'HRV', 'SDNN', 'RMSSD', 'QRS_Amp_II', 'ST_Seg_Lvl',
       'Tpeak_Tend', 'P_Amp_I', 'Q_Amp_I', 'R_Amp_I', 'S_Amp_I', 'T_Amp_I',
       'P_Amp_II', 'Q_Amp_II', 'R_Amp_II', 'S_Amp_II', 'T_Amp_II', 'P_Amp_III',
       'Q_Amp_III', 'R_Amp_III', 'S_Amp_III', 'T_Amp_III', 'P_Amp_aVR',
       'Q_Amp_aVR', 'R_Amp_aVR', 'S_Amp_aVR', 'T_Amp_aVR', 'P_Amp_aVL',
       'Q_Amp_aVL', 'R_Amp_aVL', 'S_Amp_aVL', 'T_Amp_aVL', 'P_Amp_aVF',
       'Q_Amp_aVF', 'R_Amp_aVF', 'S_Amp_aVF', 'T_Amp_aVF', 'P_Amp_V1',
       'Q_Amp_V1', 'R_Amp_V1', 'S_Amp_V1', 'T_Amp_V1', 'P_Amp_V2', 'Q_Amp_V2',
       'R_Amp_V2', 'S_Amp_V2', 'T_Amp_V2', 'P_Amp_V3', 'Q_Amp_V3', 'R_Amp_V3',
       'S_Amp_V3', 'T_Amp_V3', 'P_Amp_V4', 'Q_Amp_V4', 'R_Amp_V4', 'S_Amp_V4',
       'T_Amp_V4', 'P_Amp_V5', 'Q_Amp_V5', 'R_Amp_V5', 'S_Amp_V5', 'T_Amp_V5',
       'P_Amp_V6', 'Q_Amp_V6', 'R_Amp_V6', 'S_Amp_V6', 'T_Amp_V6'],
      dtype='object'

In [13]:
print(list(df.keys()))

['RR_Int', 'HR', 'PR_Seg', 'PR_Int', 'ST_Seg', 'QT_Int', 'QRS_Dur', 'T_Dur', 'P_Dur', 'HRV', 'SDNN', 'RMSSD', 'QRS_Amp_II', 'ST_Seg_Lvl', 'Tpeak_Tend', 'P_Amp_I', 'Q_Amp_I', 'R_Amp_I', 'S_Amp_I', 'T_Amp_I', 'P_Amp_II', 'Q_Amp_II', 'R_Amp_II', 'S_Amp_II', 'T_Amp_II', 'P_Amp_III', 'Q_Amp_III', 'R_Amp_III', 'S_Amp_III', 'T_Amp_III', 'P_Amp_aVR', 'Q_Amp_aVR', 'R_Amp_aVR', 'S_Amp_aVR', 'T_Amp_aVR', 'P_Amp_aVL', 'Q_Amp_aVL', 'R_Amp_aVL', 'S_Amp_aVL', 'T_Amp_aVL', 'P_Amp_aVF', 'Q_Amp_aVF', 'R_Amp_aVF', 'S_Amp_aVF', 'T_Amp_aVF', 'P_Amp_V1', 'Q_Amp_V1', 'R_Amp_V1', 'S_Amp_V1', 'T_Amp_V1', 'P_Amp_V2', 'Q_Amp_V2', 'R_Amp_V2', 'S_Amp_V2', 'T_Amp_V2', 'P_Amp_V3', 'Q_Amp_V3', 'R_Amp_V3', 'S_Amp_V3', 'T_Amp_V3', 'P_Amp_V4', 'Q_Amp_V4', 'R_Amp_V4', 'S_Amp_V4', 'T_Amp_V4', 'P_Amp_V5', 'Q_Amp_V5', 'R_Amp_V5', 'S_Amp_V5', 'T_Amp_V5', 'P_Amp_V6', 'Q_Amp_V6', 'R_Amp_V6', 'S_Amp_V6', 'T_Amp_V6']


In [14]:
cols_name = ["subject_id"]

In [15]:
for i in df.keys():
    cols_name.append(i)

In [16]:
len(cols_name)

76

In [17]:
# # Dont run twic
# feature_table = pd.DataFrame([[0] * 76], columns=cols_name)
# print(feature_table)
# feature_table.to_csv("/home/idies/workspace/SAFE/neurokit2_ecg_features.csv", index=False)

In [18]:
# extracted = []

In [19]:
extracted = pd.read_csv("/home/idies/workspace/SAFE/neurokit2_ecg_features.csv", usecols=["subject_id"])
extracted = extracted["subject_id"].tolist()

In [20]:
print(len(extracted))
print(extracted[-1])

21426
09:51:00_20_07_2210_12648465


In [21]:
## ECG waveform not found for advance features extraction
not_extracted = [item for item in pos_ecg_id if item not in extracted]
print(not_extracted[0:5])
print(len(not_extracted))

['15:43:00_13_01_2110_16284044', '14:12:00_28_02_2117_16982032', '09:46:00_25_06_2128_16281507', '17:14:00_19_03_2144_16289850', '08:44:00_17_01_2151_16285428']
14


In [22]:
def flatten_matrix(df, id):
    reshaped = df
    flattened = reshaped.values.flatten().tolist()
    flattened.insert(0, id)

    # Ensure that the length of 'flattened' matches the number of columns
    assert len(flattened) == len(cols_name), f"Mismatch: {len(flattened)} items in list, {len(cols_name)} columns expected."

    res = pd.DataFrame([flattened], columns=cols_name)
    return res

In [23]:
import os
import pandas as pd
import numpy as np

base_dir = '/home/idies/workspace/SAFE/ecg_preprocessed'
curr_table_path = "/home/idies/workspace/SAFE/neurokit2_ecg_features.csv"

# Assuming pos_ecg_id and extracted are properly defined
# Ensure `calculate_features`, `process_ecg`, and necessary functions like `ecg_clean`, `ecg_peaks`, `ecg_delineate` are available

# Initialize or load the existing table
if os.path.exists(curr_table_path):
    curr_table = pd.read_csv(curr_table_path)
else:
    curr_table = pd.DataFrame()

# Process each file
for filename in pos_ecg_id:
    filename = filename.strip()  # Trim whitespace
    file_path = os.path.join(base_dir, filename)  # Construct the full file path

    if os.path.exists(file_path) and filename not in extracted:
        print(f"Processing file: {file_path}")

        # Use `calculate_features` on the loaded ECG data
        features = calculate_features(file_path)
        
        features = flatten_matrix(features, filename)
        features2 = features.fillna(value=np.nan)

        # Convert the features dict to a DataFrame row
#         features_df = pd.DataFrame([features])

        # Append the new features to the current table
        curr_table = pd.concat([curr_table, features2], ignore_index=True)
        
        # Save the updated table
        curr_table.to_csv(curr_table_path, index=False)
        
        # Mark this filename as processed
        extracted.append(filename)

In [172]:
features = calculate_features("/home/idies/workspace/SAFE/ecg_preprocessed/01:47:00_08_11_2110_14717666")
print(features.shape)  # This should show (1, 75) if 75 features are expected plus 1 ID column
print(features.head())  # This will show the top rows of the DataFrame

(1, 76)
  subject_id RR_Int    HR PR_Seg PR_Int ST_Seg QT_Int QRS_Dur T_Dur P_Dur  \
0       <NA>   <NA>  <NA>   <NA>   <NA>   <NA>   <NA>    <NA>  <NA>  <NA>   

   ... P_Amp_V5 Q_Amp_V5 R_Amp_V5 S_Amp_V5 T_Amp_V5 P_Amp_V6 Q_Amp_V6  \
0  ...     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>     <NA>   

  R_Amp_V6 S_Amp_V6 T_Amp_V6  
0     <NA>     <NA>     <NA>  

[1 rows x 76 columns]


/home/idies/miniconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/idies/miniconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/idies/miniconda3/lib/python3.8/site-packages/neurokit2/signal/signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(


In [170]:
features = calculate_features("/home/idies/workspace/SAFE/ecg_preprocessed/15:21:00_07_11_2110_10205294")
print(features.shape)  # This should show (1, 75) if 75 features are expected plus 1 ID column
print(features.head())  # This will show the top rows of the DataFrame

(1, 75)
   RR_Int  HR  PR_Seg  PR_Int  ST_Seg  QT_Int  QRS_Dur  T_Dur  P_Dur  HRV  \
0     NaN NaN     NaN     NaN     NaN     NaN      NaN    NaN    NaN  NaN   

   ...  P_Amp_V5  Q_Amp_V5  R_Amp_V5  S_Amp_V5  T_Amp_V5  P_Amp_V6  Q_Amp_V6  \
0  ...       NaN       NaN       NaN       NaN       NaN       NaN       NaN   

   R_Amp_V6  S_Amp_V6  T_Amp_V6  
0       NaN       NaN       NaN  

[1 rows x 75 columns]


/home/idies/miniconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/idies/miniconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/idies/miniconda3/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/idies/miniconda3/lib/python3.8/site-packages/numpy/core/_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/home/idies/miniconda3/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
